# COSC 424/525 Homework 4
In this homework you will learn about building convolutional neural networks, residual networks, and recurrent neural networks. The main objectives of the homework is to reinforce the theory discussed in class by building such architectures from scratch and apply some of these methods to real world problems.

**General Instructions**
1. All coding should be done in Python 3
2. Always vectorize your code when possible
3. Create a Code and Markdown cells after each subtask to test and document your progress.
4. Comment your code thoroughly
5. Use a separate write-up Word file to document the experiments specified below.
5. Export your notebook and Word file in PDF format. Make sure that your PDF contains all notebook output.
6. Submit the PDF files and your Jupyter Notebook.

**Detailed Instructions**
1. CNN Step by Step [Points 40]
2. CNN Application [Points 20]
3. ResNet50 Implementation [Points 20]
4. RNN Application [Points 20]

# 3. Residual Networks: RESNET

We will build a ResNet50 architecture and compare our implementation to PyTorch built-in implementation.

Credits: [Karunesh Upadhyay](https://medium.com/@karuneshu21/how-to-resnet-in-pytorch-9acb01f36cf5)

[ResNet Paper Walkthrough](https://medium.com/@karuneshu21/resnet-paper-walkthrough-b7f3bdba55f0)

The components of the Residual Block are visualized in the following diagram:

<img src="images/resblock.png" style="width:400px;height:300px;">

At the end of the assignment, you will know how to build bottleneck modules and skip connections. You will also learn how to display text summaries and graph-based visualizations of the network components.

In [45]:
# PyTorch
import torch
import torch.nn as nn

# Display Image
from IPython.display import Image

# Visualization
import torchvision

from torchsummary import summary
# Comment out if you don't want to visualize network using a graph 
from torchviz import make_dot # this may require pip install torchviz and installation of graphviz in your system (e.g., brew install graphviz)

# Set seeds for a reproducible run
import random

%matplotlib inline

# Seeds
seed_num = 4
random.seed(seed_num)
torch.manual_seed(seed_num)

## 3.1 Bottleneck Module

You will build a functional bottleneck module using the PyTorch API. Instead of using the sequential module as in Part 2, you will define the model components inside the `__init__` function and then, define the forward propagation steps in the `forward` function.

For this exercise the `__init__` function will be given to you and you will define the forward propagation steps using the defined components.

Follow the instructions in the cell.

In [46]:
class Bottleneck(nn.Module):

    def __init__(self,in_channels,intermediate_channels,expansion,is_Bottleneck,stride):
        
        """
        Creates a Bottleneck with conv 1x1->3x3->1x1 layers.
        
        Note:
          1. Addition of feature maps occur at just before the final ReLU with the input feature maps
          2. if input size is different from output, select projected mapping or else identity mapping.
          3. if is_Bottleneck=False (3x3->3x3) are used else (1x1->3x3->1x1). Bottleneck is required for resnet-50/101/152
        Args:
            in_channels (int) : input channels to the Bottleneck
            intermediate_channels (int) : number of channels to 3x3 conv 
            expansion (int) : factor by which the input #channels are increased
            stride (int) : stride applied in the 3x3 conv. 2 for first Bottleneck of the block and 1 for remaining

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """

        super(Bottleneck,self).__init__()

        self.expansion = expansion
        self.in_channels = in_channels
        self.intermediate_channels = intermediate_channels
        self.is_Bottleneck = is_Bottleneck
        
        # i.e. if dim(x) == dim(F) => Identity function
        if self.in_channels==self.intermediate_channels*self.expansion:
            self.identity = True
        else:
            self.identity = False
            projection_layer = []
            projection_layer.append(nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels*self.expansion, kernel_size=1, stride=stride, padding=0, bias=False ))
            projection_layer.append(nn.BatchNorm2d(self.intermediate_channels*self.expansion))
            # Only conv->BN and no ReLU
            # projection_layer.append(nn.ReLU())
            self.projection = nn.Sequential(*projection_layer)

        # commonly used relu
        self.relu = nn.ReLU()

        # is_Bottleneck = True for all ResNet 50+
        if self.is_Bottleneck:

            # bottleneck
            # 1x1
            self.conv1_1x1 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False )
            self.batchnorm1 = nn.BatchNorm2d(self.intermediate_channels)
            
            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=stride, padding=1, bias=False )
            self.batchnorm2 = nn.BatchNorm2d(self.intermediate_channels)
            
            # 1x1
            self.conv3_1x1 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels*self.expansion, kernel_size=1, stride=1, padding=0, bias=False )
            self.batchnorm3 = nn.BatchNorm2d( self.intermediate_channels*self.expansion )
        
        else:
            # basicblock
            # 3x3
            self.conv1_3x3 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=stride, padding=1, bias=False )
            self.batchnorm1 = nn.BatchNorm2d(self.intermediate_channels)
            
            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=1, padding=1, bias=False )
            self.batchnorm2 = nn.BatchNorm2d(self.intermediate_channels)

    def forward(self, x):
        # Store the input to be added before the final ReLU
        in_x = x

        if self.is_Bottleneck:
            # Bottleneck block
            # 1x1 Convolution -> BatchNorm -> ReLU
            out = self.relu(self.batchnorm1(self.conv1_1x1(x)))
            # 3x3 Convolution -> BatchNorm -> ReLU
            out = self.relu(self.batchnorm2(self.conv2_3x3(out)))
            # 1x1 Convolution -> BatchNorm
            out = self.batchnorm3(self.conv3_1x1(out))
        else:
            # Basic block
            # 3x3 Convolution -> BatchNorm -> ReLU
            out = self.relu(self.batchnorm1(self.conv1_3x3(x)))
            # 3x3 Convolution -> BatchNorm
            out = self.batchnorm2(self.conv2_3x3(out))

        # Identity or projected mapping
        if self.identity:
            # If input and output have the same dimensions, perform identity mapping
            out += in_x
        else:
            # If dimensions differ, perform projection
            out += self.projection(in_x)

        # Final ReLU
        out = self.relu(out)

        return out


# Bottleneck(64*4,64,4,stride=1)

def test_Bottleneck():
    x = torch.randn(1,64,112,112)
    model = Bottleneck(64,64,4,True,2)
    print(model(x).shape)
    del model

test_Bottleneck()

##############################################
# Expected output: Expected output "torch.Size([1, 256, 56, 56])"

torch.Size([1, 256, 56, 56])


## 3.2 Build ResNet Model

You will build the forward propagation sequence for a ResNet50 model. 

<img src="images/ResNetBlocks.png" style="width:600px;height:300px;">


For this exercise the `__init__` function will be given to you and you will define the forward propagation steps using the defined components.

In [47]:
class ResNet(nn.Module):

    def __init__(self, resnet_variant,in_channels,num_classes):
        """
        Creates the ResNet architecture based on the provided variant. 18/34/50/101 etc.
        Based on the input parameters, define the channels list, repeatition list along with expansion factor(4) and stride(3/1)
        using _make_blocks method, create a sequence of multiple Bottlenecks
        Average Pool at the end before the FC layer 

        Args:
            resnet_variant (list) : eg. [[64,128,256,512],[3,4,6,3],4,True]
            in_channels (int) : image channels (3)
            num_classes (int) : output #classes 

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """
        super(ResNet,self).__init__()
        self.channels_list = resnet_variant[0]
        self.repeatition_list = resnet_variant[1]
        self.expansion = resnet_variant[2]
        self.is_Bottleneck = resnet_variant[3]

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False )
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.block1 = self._make_blocks( 64 , self.channels_list[0], self.repeatition_list[0], self.expansion, self.is_Bottleneck, stride=1 )
        self.block2 = self._make_blocks( self.channels_list[0]*self.expansion , self.channels_list[1], self.repeatition_list[1], self.expansion, self.is_Bottleneck, stride=2 )
        self.block3 = self._make_blocks( self.channels_list[1]*self.expansion , self.channels_list[2], self.repeatition_list[2], self.expansion, self.is_Bottleneck, stride=2 )
        self.block4 = self._make_blocks( self.channels_list[2]*self.expansion , self.channels_list[3], self.repeatition_list[3], self.expansion, self.is_Bottleneck, stride=2 )

        self.average_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear( self.channels_list[3]*self.expansion , num_classes)



    def forward(self,x):
        ################################
        ## YOUR CODE STARTS HERE
        # CONV1->BN->ReLU->MAXPOOL
        x = self.maxpool(self.relu(self.batchnorm1(self.conv1(x))))
        # Blocks 1 through 4
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        # Average Pool
        x = self.average_pool(x)
        # Flatten for fully connected layer
        x = x.view(x.size(0), -1)
        # Fully Connected layer
        x = self.fc1(x)
        ## YOUR CODE ENDS HERE
        #############################
        return x

    def _make_blocks(self,in_channels,intermediate_channels,num_repeat, expansion, is_Bottleneck, stride):
        
        """
        Args:
            in_channels : #channels of the Bottleneck input
            intermediate_channels : #channels of the 3x3 in the Bottleneck
            num_repeat : #Bottlenecks in the block
            expansion : factor by which intermediate_channels are multiplied to create the output channels
            is_Bottleneck : status if Bottleneck in required
            stride : stride to be used in the first Bottleneck conv 3x3

        Attributes:
            Sequence of Bottleneck layers

        """
        layers = [] 

        layers.append(Bottleneck(in_channels,intermediate_channels,expansion,is_Bottleneck,stride=stride))
        for num in range(1,num_repeat):
            layers.append(Bottleneck(intermediate_channels*expansion,intermediate_channels,expansion,is_Bottleneck,stride=1))

        return nn.Sequential(*layers)


def test_ResNet(params):
    model = ResNet( params , in_channels=3, num_classes=1000)
    x = torch.randn(1,3,224,224)
    torch.no_grad()
    output = model(x)
    print(output.shape)
    return model

# resnetX = (Num of channels, repetition, Bottleneck_expansion , Bottleneck_layer)
model_parameters={}
model_parameters['resnet50'] = ([64,128,256,512],[3,4,6,3],4,True)
model_parameters['resnet101'] = ([64,128,256,512],[3,4,23,3],4,True)
model_parameters['resnet152'] = ([64,128,256,512],[3,8,36,3],4,True)
architecture = 'resnet50'
model = test_ResNet(model_parameters[architecture])

##############################################
# Expected output: "torch.Size([1, 1000])"

torch.Size([1, 1000])


## 3.3 Compare PyTorch ResNet50 to our Model

### 3.3.1 Show text and graph-based summaries of our network.

In [48]:

display_imgs = True
summary(model, (3, 224, 224))

if display_imgs:
    #######################################
    # Optional: Comment out, if you don't want to visualize using torchviz
    x = torch.randn(1,3,224,224)
    y = model(x)
    dot = make_dot(y, params=dict(model.named_parameters()))
    dot.format = 'png'
    dot.render('my_model_graph')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,


### 3.3.2 Show text and graph-based summaries of PyTorch Torchvision network.

In [49]:
from torchvision.models import resnet34,resnet50,resnet18,resnet101,resnet152,ResNet50_Weights
torchvision_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
summary(torchvision_model, (3, 224, 224))

if display_imgs:
    #######################################
    # Optional: Comment out, if you don't want to visualize using torchviz
    x = torch.randn(1,3,224,224)
    y = torchvision_model(x)
    dot = make_dot(y, params=dict(torchvision_model.named_parameters()))
    dot.format = 'png'
    dot.render('torch_model_graph')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,


### 3.3.3 Display model graphs side by side.

In [50]:
from IPython.display import HTML

# Create HTML content to display images side by side
html_content = f"""
<div style='float: left; padding-right: 10px;'>
    <p>Our Model:</p>
    <img src='my_model_graph.png' width='500'>
</div>
<div style='float: left; padding-left: 10px;'>
    <p>Torchvision Model:</p>
    <img src='torch_model_graph.png' width='500'>
</div>
"""

if display_imgs:
    # Display HTML
    display(HTML(html_content))